In [2]:
# type: ignore

import requests
import numpy as np

import pandas as pd 
import geopandas as gpd

import pyproj
from shapely.ops import transform

import shapely
from shapely import Point

# import boto3
import os
import django

pd.set_option('display.precision', 2)

wdwanted = '/Users/cg/Dropbox/code/Python/water/app/backend/'
os.chdir(wdwanted)

env_file = '../.env'
file_content = open(env_file).read()
lines = file_content.splitlines()
for line in lines:
    parts = line.split('=')
    if len(parts) == 2:
        key, value = parts
        os.environ[key] = value
        
DB_USED = 'rdstunnel'
os.environ['DB_USED']=DB_USED

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "water.settings.dev")
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
_ = django.setup()
from water.models import ReservoirState, Reservoir, ReservoirStateSerializer, RainFall

from water.utils import parse as p
from django.db.models import Count
from water.utils import data

# Reload the data module
import importlib
importlib.reload(data)

<module 'water.utils.data' from '/Users/cg/Dropbox/code/Python/water/app/backend/water/utils/data.py'>

In [14]:
folder_data = "water/data_raw"
filename_data = f"{folder_data}/all_parsed_cleaned.csv"

df_raw = pd.read_csv(filename_data)

def pick_yearly(df):
    df_monthly = df[df.ds.str.slice(5, 10) == "09-01"].copy()
    return df_monthly

df_year = pick_yearly(df_raw)
df_year.columns

df_year['year'] = df_year.ds.str.slice(0, 4).astype(int)

df_year.columns

Index(['ds', 'reservoir', 'province', 'rainfallsince', 'avgrainfall1971_2000',
       'capacity_hm3', 'stored_hm3', 'variation24h_hm3', 'weekchange',
       'yearchange', 'fillingpcttoday', 'weekago', 'yearago',
       'avgreserve5yrs_hm3', 'year'],
      dtype='object')

In [12]:
df_year.query('year==2023').groupby('province')[['capacity_hm3', 'stored_hm3']].sum()

,capacity_hm3,stored_hm3
province,,
almeria,222.98,31.61
cadiz,1821.00,326.11
cordoba,3248.20,486.60
granada,1126.60,290.79
huelva,1410.00,401.97
jaen,2457.90,495.29
malaga,616.80,144.01
sevilla,956.20,266.94


In [13]:
df_year.groupby(['year', 'province'])[['stored_hm3']].sum().unstack(1)

stored_hm3                                                      \
province    almeria    cadiz  cordoba granada   huelva     jaen  malaga   
year                                                                      
2012          16.83  1130.51  1848.96  669.68   913.61  1163.37  398.73   
2013          39.93  1529.39  2801.34  769.22  1110.99  1869.60  482.43   
2014          26.31  1363.40  2552.88  810.11   831.87  1792.21  431.80   
2015          21.07  1190.36  2061.51  686.78  1093.95  1469.59  375.72   
2016          13.75   651.54    38.62  450.86   295.33   424.39  169.03   
2017          23.51   531.89   358.79   76.41   144.65   408.01  171.24   
2018          21.50   683.62   552.18  510.91  1012.08   723.33  290.58   
2019          22.89   571.60   335.51  214.55   520.63   170.88  205.08   
2020          31.09   378.14   452.85  144.97   243.88   519.86  188.03   
2021           5.67   421.74   727.17  203.61   442.40   561.71  202.21   
2022          45.81   341.31   547.66  227.26   258.01   258.42  186.68   
2023          31.61   326.11   486.60  290.79   401.97   495.29  144.01   

                  
province sevilla  
year              
2012      497.60  
2013      630.40  
2014      601.67  
2015      410.10  
2016      268.65  
2017      216.21  
2018      569.80  
2019      163.74  
2020      327.72  
2021      339.74  
2022      132.01  
2023      266.94

In [17]:
sums = df_year.groupby(['year', 'province'])[['capacity_hm3']].sum().unstack(1)

sums

capacity_hm3                                                        
province      almeria   cadiz cordoba granada  huelva    jaen  malaga sevilla
year                                                                         
2012           222.98  1821.0  3248.2  1184.8  1410.0  2289.9  620.95   891.8
2013           222.98  1821.0  3248.2  1055.9  1350.5  2289.9  529.75   821.6
2014           222.98  1821.0  3248.2  1184.8   954.5  2457.9  620.95   862.4
2015           222.98  1821.0  3248.2  1184.8  1398.0  2457.9  616.80   685.6
2016           222.98  1131.0    60.3   913.8   486.2  1021.5  401.60   393.9
2017           222.98  1124.9  1186.4   276.1   193.3  1248.3  488.40   424.3
2018           222.98  1043.3  1311.6  1015.1  1367.4  1434.2  488.40   739.0
2019           222.98  1043.3  1321.5   386.2   839.7   407.7  362.70   324.9
2020           222.98   908.3  1830.6   351.4   534.7  1625.5  362.70   486.7
2021            61.68  1271.4  3248.2   775.4   827.7  2168.7  488.40   688.1
2022           222.98  1353.0  3248.2   971.1   779.2  1421.3  488.40   315.6
2023           222.98  1821.0  3248.2  1126.6  1410.0  2457.9  616.80   956.2

In [18]:
sums * 0.15

capacity_hm3                                                       
province      almeria   cadiz cordoba granada  huelva    jaen malaga sevilla
year                                                                        
2012            33.45  273.15  487.23  177.72  211.50  343.49  93.14  133.77
2013            33.45  273.15  487.23  158.39  202.57  343.49  79.46  123.24
2014            33.45  273.15  487.23  177.72  143.17  368.69  93.14  129.36
2015            33.45  273.15  487.23  177.72  209.70  368.69  92.52  102.84
2016            33.45  169.65    9.04  137.07   72.93  153.22  60.24   59.08
2017            33.45  168.73  177.96   41.41   29.00  187.24  73.26   63.64
2018            33.45  156.49  196.74  152.26  205.11  215.13  73.26  110.85
2019            33.45  156.49  198.22   57.93  125.95   61.15  54.41   48.73
2020            33.45  136.24  274.59   52.71   80.20  243.82  54.41   73.00
2021             9.25  190.71  487.23  116.31  124.16  325.30  73.26  103.22
2022            33.45  202.95  487.23  145.66  116.88  213.19  73.26   47.34
2023            33.45  273.15  487.23  168.99  211.50  368.69  92.52  143.43

In [16]:
df_year.groupby(['year', 'province'])[['capacity_hm3']].count().unstack(1)

capacity_hm3                                                 
province      almeria cadiz cordoba granada huelva jaen malaga sevilla
year                                                                  
2012                2    10      12      11      8   12      7       9
2013                2    10      12       8      7   12      5       9
2014                2    10      12      11      4   13      7       8
2015                2    10      12      11      7   13      7       8
2016                2     5       3       7      3    7      5       3
2017                2     5       6       6      2    9      5       6
2018                2     4       6       8      7    8      5       6
2019                2     4       6       6      4    5      4       4
2020                2     4       8       7      3    8      4       5
2021                1     5      12       5      3   11      5       7
2022                2     6      12       8      3    9      5       4
2023                2    10      12      10      8   13      7      10